Project by Jeremy Bouhi & Lucas Trevalinet

# Classification Document


## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import re

## STEP 1 : Grab Data

In [2]:
# We consider you had already unzip the dataset file and that this file is in the parent folder

#GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global") #Lucas' Path
GLOBAL_PATH = os.path.join("..","20180315-170000", "juri","cass","global") #Jeremy's Path

# We need to do that for excluding data from Juri_path
CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

In [3]:
ident = []
text = []
division = []
DATA_PATH = [CIVILE_PATH] # For faster test
#DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

for DIVISION_PATH in DATA_PATH :
    xml_files = list(Path(DIVISION_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)
                
            # For getting the text
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = re.sub('<[^<]+?>', '', ET.tostring(child).decode("utf-8"))
                text.append(contenu)
                
            # For getting the division    
            for child in root.iter('META_JURI_JUDI'):
                formation = re.sub('CHAMBRE|_|[0-9]', '', child.find('FORMATION').text)
                division.append(formation)
                
            

In [4]:
print ("text: ", len(text))
print ("ID: ", len(ident))
print ("division: ", len(division))

text:  59250
ID:  59250
division:  59250


## STEP 2 : Create DataFrame to manipulate easily data

In [6]:
d = {'id': ident, 'text': text, 'division': division}
df = pd.DataFrame(data = d)

#print('df', df)

df["text"]

0        \nLA COUR DE CASSATION, TROISI&#200;ME CHAMBRE...
1        \nLA COUR DE CASSATION, TROISI&#200;ME CHAMBRE...
2        \nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBRE ...
3        \nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBRE ...
4        \nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBRE ...
5        \nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBRE ...
6        \n\nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBR...
7        \nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBRE ...
8        \nLA COUR DE CASSATION, PREMI&#200;RE CHAMBRE ...
9        \nLA COUR DE CASSATION, PREMI&#200;RE CHAMBRE ...
10       \nLA COUR DE CASSATION, PREMI&#200;RE CHAMBRE ...
11       \nLA COUR DE CASSATION, TROISI&#200;ME CHAMBRE...
12       \nLA COUR DE CASSATION, TROISI&#200;ME CHAMBRE...
13       \nLA COUR DE CASSATION, TROISI&#200;ME CHAMBRE...
14       \nLA COUR DE CASSATION, TROISI&#200;ME CHAMBRE...
15       \nLA COUR DE CASSATION, DEUXI&#200;ME CHAMBRE ...
16       \nLA COUR DE CASSATION, PREMI&#200;RE CHAMBRE .

## STEP 3 : Split Data

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test= train_test_split(df["text"],df["division"], test_size = 0.2 ,random_state=42,stratify=df["division"] )
print(X_train.head())

19298    \nSur le moyen unique du pourvoi incident, qui...
31218    \nSur le moyen unique :\n   Vu l'article 2127 ...
28634    \n\n   Sur le moyen unique : \n\n   Attendu qu...
35146                                               \n\n\n
22017    \nSur la recevabilit&#233; du pourvoi contest&...
Name: text, dtype: object


In [8]:
print("Training size : ",X_train.shape[0])
print("Testing size : ",X_test.shape[0])

Training size :  47400
Testing size :  11850


## STEP 4 : Clean Data

In [28]:
# Set back special characters removed by decode(utf-8) used for grab the whole text (not until the first <br>)
def set_back_special_characters(s) : 
    return s.replace('&#233;', 'é').replace('&#201;', 'é').replace("&#234;","ê").replace("&#200;","è").replace("&#232;","è").replace("&#249;","ù").replace("&#224;","à").replace("&#251;","û").replace('&#226;', 'â').replace('&#238;', 'î').replace('&#239;','ï').replace('&#231;','ç').replace('&#244;','ô').replace('&#176;','°').replace('&#187;','»').replace('&#171;','«').replace('&#8211;','–')

In [29]:
X_train = X_train.apply(set_back_special_characters)

In [62]:
# Preprocessing step pull off all the htlm tag and "\n"
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    #text2 = text.replace('\n','').replace('\'',' ').replace(';','').replace(',','').replace('.','').replace('  ',' ')
    return text.replace('[.,\/#!$%\^&\*;:{}=\-_`~()«»°–]','').replace('\s{2,}',' ')

In [63]:
X_train = X_train.apply(preprocessor)

In [54]:
def clean_references_to_law_codes(text) : 
    return re.sub('(?<=Code )civil|de l action sociale et des familles|de l artisanat|des assurances|de l aviation civile|du cinéma et de l image animée|de commerce|des communes( de la Nouvelle-Calédonie)?|de la consommation', '', text)

In [55]:
X_train = X_train.apply(clean_references_to_law_codes)

In [65]:
def clean_words_corresponding_to_labels(text) :
    return re.sub('criminel(le)?|commercial(e)?|social(e)?|civil(e)?', '*_*', text)

In [66]:
X_train = X_train.apply(clean_words_corresponding_to_labels)

In [56]:
# Set to lower case
def to_lower_case(s) :
    return s.lower()

In [57]:
X_train = X_train.apply(to_lower_case)

In [67]:
X_train[4]

'la cour de cassation deuxième chambre *_* a rendu l arrêt suivant : sur le premier moyen pris en sa seconde branche : vu l article 13 du titre ii de la loi des 16-24 août 1790 et le décret du 16 fructidor an ii et l article l 142-1 du code de la sécurité *_* ensemble les articles l 254-1 et l 254-2 du code dans leur rédaction applicable à la date des soins litigieuxattendu que les litiges nés de la facturation aux caisses d assurance maladie par les établissements de santé de la part des dépenses prises en charge par l etat pour les soins dispensés au titre des soins urgents dans les conditions prévues par les deux derniers de ces textes se rapportent à l attribution des prestations d aide *_* de l etat et ne sont pas dès lors au nombre des différends auxquels donne lieu au sens du troisième l application des législations et réglementations de sécurité *_* et de mutualité *_* agricoleattendu selon le jugement attaqué rendu en dernier ressort qu ayant dispensé au titre des soins urgent

In [53]:
#Stop_words :
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def clean_data(text):
    
    french_stopwords = set(stopwords.words('french'))
    tokens = word_tokenize(text, language='french')
    
    content_tokens = ""
    for token in tokens:
        if token not in french_stopwords:
            content_tokens += token
            content_tokens += " "
    return(content_tokens) 

In [ ]:
X_train = X_train.apply(clean_data)